In [1]:
from data_classes.IMDBLightningDataModule import IMDBLightningDataModule
from models.ClassifierSystem import LightningClassifier
from data_classes.pretrained_embeddings import get_pretrained_embeddings
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

In [2]:
vocab, vectors = get_pretrained_embeddings()

In [3]:
imdb_data = IMDBLightningDataModule(vocab)

In [36]:
classifier = LightningClassifier(vocab=vocab, vectors=vectors, embedding_size=300)

In [37]:
logger = TensorBoardLogger('exp_logs', name='lstm')

In [38]:
trainer = Trainer(max_epochs=10, gpus=1, auto_select_gpus=True, auto_scale_batch_size=False, auto_lr_find=True, logger=[logger])
# , track_grad_norm=2, accumulate_grad_batches=8

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [39]:
trainer.test(classifier, imdb_data)

C:\Users\franc\Anaconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\franc\Anaconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing:   0%|          | 3/782 [00:00<00:32, 24.26it/s]

c:\Users\franc\Documents\inge_master_2\web_and_text_analysis\sentiment_analysis\src\data_classes\IMDB.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return text_pipeline(self.data[idx][1]), torch.tensor(labels, dtype=torch.long)


Testing: 100%|██████████| 782/782 [00:46<00:00, 16.41it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Acc': 0.5122399926185608, 'Test Loss': 0.6921125650405884}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 782/782 [00:46<00:00, 16.89it/s]


C:\Users\franc\Anaconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


[{'Test Loss': 0.6921125650405884, 'Test Acc': 0.5122399926185608}]

In [40]:
trainer.tune(classifier, imdb_data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | LSTM             | 3.2 M 
1 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
161 K     Trainable params
3.0 M     Non-trainable params
3.2 M     Total params
12.644    Total estimated model params size (MB)
C:\Users\franc\Anaconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\franc\Anaconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be

{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x1a76663f640>}

In [41]:
classifier.batch_size

32

In [42]:
classifier.learning_rate

0.002754228703338169

In [43]:
trainer.fit(classifier, imdb_data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | LSTM             | 3.2 M 
1 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
161 K     Trainable params
3.0 M     Non-trainable params
3.2 M     Total params
12.644    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 783/783 [01:32<00:00,  7.42it/s, loss=0.0609, v_num=5]


In [44]:
trainer.test(classifier, imdb_data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 782/782 [00:49<00:00, 14.91it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Acc': 0.8784000277519226, 'Test Loss': 0.4879884421825409}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 782/782 [00:49<00:00, 15.82it/s]


[{'Test Loss': 0.4879884421825409, 'Test Acc': 0.8784000277519226}]